In [1]:
import arcpy
import time
import os
import csv

# -----------------------------
# Environment Setup
# -----------------------------
arcpy.env.overwriteOutput = True
print("🚀 Starting ArcPy Deep Learning training (robust, unfrozen backbone)...\n")

# -----------------------------
# Paths
# -----------------------------
training_data_path = r"C:\Users\ss2596\Documents\njoko training\hansen_2022_2024_fullband"
model_output_path = r"C:\Users\ss2596\Documents\Njoko_model\hansen_2022_2024_FullBand_DLmodel"
os.makedirs(model_output_path, exist_ok=True)

metrics_file = os.path.join(model_output_path, "training_metrics.csv")
if not os.path.exists(metrics_file):
    with open(metrics_file, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["epoch", "messages"])

# -----------------------------
# Data Sanity Checks
# -----------------------------
images_path = os.path.join(training_data_path, "images")
labels_path = os.path.join(training_data_path, "labels")

num_images = len(os.listdir(images_path)) if os.path.exists(images_path) else 0
num_labels = len(os.listdir(labels_path)) if os.path.exists(labels_path) else 0

print(f"Found {num_images} images and {num_labels} labels in the training folder.")

if num_images == 0 or num_labels == 0:
    raise RuntimeError("Training folder is missing images or labels. Check your folder structure!")

# -----------------------------
# Training Parameters
# -----------------------------
max_epochs = 20
batch_size = 4           # Reduced batch size for memory safety
chip_size = 128          # Smaller chip size to reduce memory usage
model_type = "ChangeDetector"
backbone_model = "RESNET50"

# -----------------------------
# Launch Training
# -----------------------------
print("🧠 Launching training job...\n")

result = arcpy.ia.TrainDeepLearningModel(
    training_data_path,               # in_training_data
    model_output_path,                # out_model
    max_epochs,                       # max_epochs
    model_type,                        # model_type
    batch_size,                        # batch_size
    {"attention_type": "PAM"},         # model_arguments
    "",                                # learning_rate
    backbone_model,                     # backbone_model
    "",                                 # pretrained_model
    10,                                 # validation_percent
    "VALID_LOSS",                       # stop_training
    "UNFREEZE_MODEL",                   # freeze (unfrozen backbone)
    "DEFAULT",                           # augmentation
    "",                                  # augmentation_parameters
    chip_size,                           # chip_size
    "",                                  # resize_to
    "ALL_RANDOM",                        # weight_init_scheme
    "VALID_LOSS"                         # monitor_metric
)

# -----------------------------
# Polling Loop for Live Logging
# -----------------------------
epoch = 0
while result.status < 4:  # 4 = Succeeded/Failed
    messages = result.getMessages(0)
    print(messages, flush=True)

    # Save messages to CSV per poll
    epoch += 1
    with open(metrics_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([epoch, messages.strip()])

    time.sleep(30)  # Wait before next check

# -----------------------------
# Final Messages
# -----------------------------
print("\n📊 Final Training Messages:\n")
print(result.getMessages())

print("\n✅ Training completed.")
print(f"Checkpoints and logs saved in: {model_output_path}")
print("-------------------------------------------------------------")

🚀 Starting ArcPy Deep Learning training (backbone unfrozen)...

🧠 Launching training job...



RuntimeError: Object: Error in executing tool